In [1]:
import os
import time
import warnings
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
from timm import create_model

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Load CSV and define paths ===
train_dir = "data/train_data"
test_dir = "data/test_data"
train_df = pd.read_csv('data/Train.csv')
test_df = pd.read_csv('data/Test.csv')
train_df['ID_path'] = train_df['ID'].apply(lambda x: os.path.join(train_dir, f"{x}.npy"))
test_df['ID_path'] = test_df['ID'].apply(lambda x: os.path.join(test_dir, f"{x}.npy"))

train = train_df[['ID_path', 'label']]#.copy()
train.columns = ['image', 'label']

# === Train/Validation Split ===
train_set, valid_set = train_test_split(train, stratify=train['label'], test_size=0.2, random_state=42)


In [13]:
test = test_df[['ID_path']]
test.columns = ['image']
test.head()

,image
0,data/test_data/ID_ICB8K9.npy
1,data/test_data/ID_2D4AOJ.npy
2,data/test_data/ID_2TVPI0.npy
3,data/test_data/ID_E05WIK.npy
4,data/test_data/ID_KKFDJO.npy


In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image

def load_and_normalize_npy_image(path):
    img = np.load(path).astype(np.float32)
    img = (img - img.min((0, 1))) / (img.max((0, 1)) - img.min((0, 1)) + 1e-5)
    return img

def convert_npy_to_jpg(df, output_dir='data/jpg_data'):
    os.makedirs(output_dir, exist_ok=True)
    new_data = []

    for idx, row in df.iterrows():
        npy_path = row['image']
        label = row['label']
        base_name = os.path.splitext(os.path.basename(npy_path))[0]

        try:
            image_array = load_and_normalize_npy_image(npy_path)
            image_uint8 = (image_array * 255).clip(0, 255).astype(np.uint8)

            # Try Option 1: Use first 3 channels as RGB
            try:
                if image_uint8.ndim == 2:
                    img = Image.fromarray(image_uint8, mode='L')
                    jpg_path = os.path.join(output_dir, f"{base_name}.jpg")
                    img.save(jpg_path, format='JPEG')
                    new_data.append({'image': jpg_path, 'label': label})

                elif image_uint8.shape[2] >= 3:
                    img = Image.fromarray(image_uint8[:, :, :3], mode='RGB')
                    jpg_path = os.path.join(output_dir, f"{base_name}.jpg")
                    img.save(jpg_path, format='JPEG')
                    new_data.append({'image': jpg_path, 'label': label})

                else:
                    raise ValueError("Too few channels for RGB")

            except Exception as e1:
                # Try Option 2: Average all channels to grayscale
                try:
                    avg_img = np.mean(image_uint8, axis=2).astype(np.uint8)
                    img = Image.fromarray(avg_img, mode='L')
                    jpg_path = os.path.join(output_dir, f"{base_name}_avg.jpg")
                    img.save(jpg_path, format='JPEG')
                    new_data.append({'image': jpg_path, 'label': label})

                except Exception as e2:
                    # Try Option 3: Save each channel as a separate grayscale image
                    try:
                        for ch in range(image_uint8.shape[2]):
                            ch_img = Image.fromarray(image_uint8[:, :, ch], mode='L')
                            jpg_path = os.path.join(output_dir, f"{base_name}_ch{ch:02d}.jpg")
                            ch_img.save(jpg_path, format='JPEG')
                            new_data.append({'image': jpg_path, 'label': label})

                    except Exception as e3:
                        print(f"[SKIPPED] {npy_path} — All save attempts failed.")
                        continue

        except Exception as outer:
            print(f"[ERROR] Failed to load/process {npy_path}: {outer}")
            continue

    return pd.DataFrame(new_data)


In [15]:
test = test_df[['ID_path']]
test.columns = ['image']
test_jpg = convert_npy_to_jpg(test)
test.head(), test_jpg.head()

KeyError: 'label'

In [ ]:
train_jpg = convert_npy_to_jpg(train_set)
valid_jpg = convert_npy_to_jpg(valid_set)
# account for the non labels ones 
# test_jpg https://auto.gluon.ai/stable/tutorials/multimodal/image_prediction/beginner_image_cls.html

In [4]:
train_jpg

,image,label
0,data/jpg_data/ID_V986RM.jpg,0
1,data/jpg_data/ID_32HCM2.jpg,0
2,data/jpg_data/ID_ZSGZJ3.jpg,0
3,data/jpg_data/ID_5HS2RZ.jpg,0
4,data/jpg_data/ID_GV66RY.jpg,0
...,...,...
5712,data/jpg_data/ID_XP3W0M.jpg,0
5713,data/jpg_data/ID_Y66V3E.jpg,0
5714,data/jpg_data/ID_WOXBJP.jpg,0
5715,data/jpg_data/ID_IVT5K1.jpg,0


In [5]:
from autogluon.multimodal import MultiModalPredictor
import uuid
model_path = f"./tmp/{uuid.uuid4().hex}-automm_shopee"
predictor = MultiModalPredictor(label="label", path=model_path)
predictor.fit(
    train_data=train_jpg,
    time_limit=30*100, # seconds
    presets ='best_quality'
)

2025-06-03 23:24:49.510794: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-03 23:24:49.518445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748989489.526812  511011 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748989489.529378  511011 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748989489.536500  511011 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 20: 'val_roc_auc' reached 0.84786 (best 0.84786), saving model to '/home/tdx/Documents/CodeX/Landslides/tmp/897804a501944c6fa943dc32de81f4ee-automm_shopee/epoch=0-step=20.ckpt' as top 3


...

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/home/tdx/Documents/CodeX/Landslides/tmp/897804a501944c6fa943dc32de81f4ee-automm_shopee")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




In [6]:
scores = predictor.evaluate(valid_jpg, metrics=["accuracy"])
print('Top-1 test acc: %.3f' % scores["accuracy"])

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

Top-1 test acc: 0.939


In [34]:
scores

{'accuracy': 0.8853146853146853}

In [8]:
image_path = test.iloc[0]['image']
from IPython.display import Image, display
pil_img = Image(filename=image_path)
display(pil_img)

NameError: name 'test' is not defined